In [2]:
# Testando instalação do PySpark
# 15/10/2025

# Configuração inicial
import os
import sys

python_path = sys.executable
os.environ['PYSPARK_PYTHON'] = python_path
os.environ['PYSPARK_DRIVER_PYTHON'] = python_path
os.environ['SPARK_LOCAL_HOSTNAME'] = 'localhost'

print("Configuração do ambiente:")
print(f"Python: {sys.version}")
print(f"Executável: {python_path}")

from pyspark.sql import SparkSession
import pandas as pd
print("Bibliotecas importadas")

# ============================================

# Criando Spark Session
os.environ['SPARK_LOCAL_HOSTNAME'] = 'localhost'

spark = SparkSession.builder \
    .appName("TestePySpark") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

print(f"Spark {spark.version} iniciado")
print(f"App: {spark.sparkContext.appName}")

# ============================================

# Criando dados de teste
dados = [
    (1, "João Silva", 28, "São Paulo", 5500.00),
    (2, "Maria Santos", 34, "Rio de Janeiro", 6200.00),
    (3, "Pedro Oliveira", 25, "Belo Horizonte", 4800.00),
    (4, "Ana Costa", 41, "Curitiba", 7100.00),
    (5, "Carlos Souza", 29, "Porto Alegre", 5200.00),
    (6, "Julia Mendes", 37, "Brasília", 6800.00),
    (7, "Lucas Ferreira", 31, "Fortaleza", 5400.00),
    (8, "Fernanda Lima", 26, "Salvador", 4900.00),
]

colunas = ["id", "nome", "idade", "cidade", "salario"]
df_spark = spark.createDataFrame(dados, colunas)

print(f"DataFrame criado - {df_spark.count()} registros")
print(f"Colunas: {df_spark.columns}")

# ============================================

# Visualizando os dados
df_spark.show()

# ============================================

# Estatísticas básicas
df_spark.describe().show()

# ============================================

# Salvando CSV para testar leitura depois
os.makedirs("../data", exist_ok=True)

csv_path = "../data/funcionarios_teste.csv"
df_spark.coalesce(1).write.mode("overwrite").csv(csv_path, header=True)

print(f"CSV salvo: {csv_path}")

# ============================================

# Testando leitura do CSV
df_lido = spark.read.csv(
    csv_path,
    header=True,
    inferSchema=True
)

print(f"CSV lido - {df_lido.count()} registros")
df_lido.show()

# ============================================

# Algumas operações básicas

# Filtrar por idade
print("Funcionários com mais de 30 anos:")
df_lido.filter(df_lido.idade > 30).show()

# Selecionar colunas
print("Nome e salário:")
df_lido.select("nome", "salario").show()

# Ordenar
print("Ordenado por salário:")
df_lido.orderBy(df_lido.salario.desc()).show()

# Agrupar
print("Salário médio por cidade:")
df_lido.groupBy("cidade").avg("salario").show()

# ============================================

# Convertendo para Pandas
df_pandas = df_lido.toPandas()
print(f"Convertido para Pandas - tipo: {type(df_pandas)}")
print(df_pandas.head())

# ============================================

# Schema
df_lido.printSchema()

# ============================================

# Informações do contexto
print(f"Master: {spark.sparkContext.master}")
print(f"Paralelismo: {spark.sparkContext.defaultParallelism}")
print(f"Usuário: {spark.sparkContext.sparkUser()}")

# ============================================

# Testando SQL
df_lido.createOrReplaceTempView("funcionarios")

resultado_sql = spark.sql("""
    SELECT 
        cidade,
        COUNT(*) as total_funcionarios,
        AVG(salario) as salario_medio,
        MAX(salario) as maior_salario
    FROM funcionarios
    GROUP BY cidade
    ORDER BY salario_medio DESC
""")

print("Query SQL executada:")
resultado_sql.show()

# ============================================

print("Testes concluídos - PySpark funcionando ok")

Configuração do ambiente:
Python: 3.10.0 (tags/v3.10.0:b494f59, Oct  4 2021, 19:00:18) [MSC v.1929 64 bit (AMD64)]
Executável: C:\Users\Gabriel\Documents\ml-data-studying-projects\venv_spark\Scripts\python.exe
Bibliotecas importadas
Spark 4.0.1 iniciado
App: TestePySpark
DataFrame criado - 8 registros
Colunas: ['id', 'nome', 'idade', 'cidade', 'salario']
+---+--------------+-----+--------------+-------+
| id|          nome|idade|        cidade|salario|
+---+--------------+-----+--------------+-------+
|  1|    João Silva|   28|     São Paulo| 5500.0|
|  2|  Maria Santos|   34|Rio de Janeiro| 6200.0|
|  3|Pedro Oliveira|   25|Belo Horizonte| 4800.0|
|  4|     Ana Costa|   41|      Curitiba| 7100.0|
|  5|  Carlos Souza|   29|  Porto Alegre| 5200.0|
|  6|  Julia Mendes|   37|      Brasília| 6800.0|
|  7|Lucas Ferreira|   31|     Fortaleza| 5400.0|
|  8| Fernanda Lima|   26|      Salvador| 4900.0|
+---+--------------+-----+--------------+-------+

+-------+-----------------+--------------+